# Phase 4 — Competitor Analysis & Investment Verdict
### BNPL Under the Microscope

**Objective:** Benchmark Klarna against AFRM, PYPL, and SQ across financial metrics,
and synthesize the full analysis into a structured investment verdict.

**Key Questions We Answer:**
1. Which BNPL player has the best risk-adjusted fundamentals?
2. How do valuation multiples compare across the competitive set?
3. What does the stock performance correlation tell us about sector dynamics?
4. Does the data support a Bull, Bear, or Base case for KLAR at current prices?

**Sources:** Klarna SEC filings, CFPB reports, NY Fed, CB Insights, Bloomberg, CNBC

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('dark_background')

COLORS = {
    'gold':    '#E8C56D',
    'red':     '#FF5B5B',
    'green':   '#6FCF97',
    'blue':    '#4A9EFF',
    'orange':  '#C45C3A',
    'muted':   '#7A7A8A',
    'bg':      '#0A0A0C',
    'surface': '#111116',
}

TICKER_COLORS = {
    'KLAR': COLORS['gold'],
    'AFRM': COLORS['green'],
    'PYPL': COLORS['blue'],
    'SQ':   COLORS['orange'],
}

plt.rcParams.update({
    'figure.facecolor': COLORS['bg'],
    'axes.facecolor':   COLORS['surface'],
    'axes.edgecolor':   '#2A2A35',
    'axes.labelcolor':  '#7A7A8A',
    'xtick.color':      '#7A7A8A',
    'ytick.color':      '#7A7A8A',
    'grid.color':       '#1E1E28',
    'grid.linestyle':   '--',
    'grid.linewidth':   0.5,
    'font.family':      'monospace',
    'font.size':        10,
    'figure.dpi':       150,
})

RAW_DIR  = '../data/raw'
PROC_DIR = '../data/processed'
VIS_DIR  = '../visuals'
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

competitors = pd.read_csv(f'{RAW_DIR}/competitor_snapshot.csv')
prices_raw  = pd.read_csv(f'{RAW_DIR}/stock_prices_raw.csv', index_col=0, parse_dates=True)
prices_norm = pd.read_csv(f'{RAW_DIR}/stock_prices_normalised.csv', index_col=0, parse_dates=True)

print('Data loaded successfully.')
competitors

## 4.1 — Valuation Multiples Comparison

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
comp = competitors[competitors['ticker'].isin(['KLAR', 'AFRM', 'PYPL', 'SQ'])].copy()
colors = [TICKER_COLORS[t] for t in comp['ticker']]

# Plot 1: P/S Ratio
ax = axes[0]
bars = ax.bar(comp['ticker'], comp['ps_ratio'], color=colors, width=0.55, zorder=3)
for bar, val in zip(bars, comp['ps_ratio']):
    ax.text(bar.get_x() + bar.get_width()/2, val + 0.03,
            f'{val:.2f}x', ha='center', va='bottom', color='white',
            fontsize=11, fontweight='bold')
ax.set_title('Price / Revenue Multiple (P/S)', color='white', fontsize=11)
ax.set_ylabel('P/S Ratio')
ax.grid(True, axis='y', alpha=0.4)
ax.set_ylim(0, max(comp['ps_ratio']) * 1.25)

# Plot 2: Revenue Growth
ax = axes[1]
bars = ax.bar(comp['ticker'], comp['revenue_growth_pct'], color=colors, width=0.55, zorder=3)
for bar, val in zip(bars, comp['revenue_growth_pct']):
    ax.text(bar.get_x() + bar.get_width()/2, val + 0.3,
            f'{val}%', ha='center', va='bottom', color='white',
            fontsize=11, fontweight='bold')
ax.set_title('Revenue Growth YoY (%)', color='white', fontsize=11)
ax.set_ylabel('Growth %')
ax.grid(True, axis='y', alpha=0.4)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())

# Plot 3: Stock return since KLAR IPO
ax = axes[2]
ret_colors = [COLORS['green'] if v >= 0 else COLORS['red']
              for v in comp['stock_return_since_klar_ipo']]
bars = ax.bar(comp['ticker'], comp['stock_return_since_klar_ipo'],
              color=ret_colors, width=0.55, zorder=3)
ax.axhline(0, color='#2A2A35', linewidth=1)
for bar, val in zip(bars, comp['stock_return_since_klar_ipo']):
    ypos = val + 0.5 if val >= 0 else val - 2.5
    ax.text(bar.get_x() + bar.get_width()/2, ypos,
            f'{val:+}%', ha='center', va='bottom', color='white',
            fontsize=11, fontweight='bold')
ax.set_title('Stock Return Since KLAR IPO\n(Sep 10, 2025 baseline)', color='white', fontsize=11)
ax.set_ylabel('Return (%)')
ax.grid(True, axis='y', alpha=0.4)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())

plt.suptitle('BNPL Competitor Comparison — Valuation, Growth & Performance',
             color='white', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(f'{VIS_DIR}/09_competitor_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: visuals/09_competitor_comparison.png')

## 4.2 — Stock Correlation Heatmap

In [ ]:
returns = prices_raw.pct_change().dropna()
corr_matrix = returns[['KLAR', 'AFRM', 'PYPL', 'SQ', '^GSPC']].corr()

fig, ax = plt.subplots(figsize=(9, 7))

sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=2,
    linecolor=COLORS['bg'],
    annot_kws={'size': 13, 'color': 'white', 'fontweight': 'bold'},
    ax=ax,
    cbar_kws={'shrink': 0.8, 'label': 'Correlation Coefficient'}
)

ticker_labels = {
    'KLAR':  'KLAR (Klarna)',
    'AFRM':  'AFRM (Affirm)',
    'PYPL':  'PYPL (PayPal)',
    'SQ':    'SQ (Block)',
    '^GSPC': 'S&P 500'
}

ax.set_xticklabels(
    [ticker_labels[t] for t in corr_matrix.columns],
    rotation=30, ha='right', color=COLORS['muted'], fontsize=10
)
ax.set_yticklabels(
    [ticker_labels[t] for t in corr_matrix.index],
    rotation=0, color=COLORS['muted'], fontsize=10
)
ax.set_title('Daily Returns Correlation Matrix — Post-IPO Period',
             color='white', fontsize=13, pad=16, loc='left')

plt.tight_layout()
plt.savefig(f'{VIS_DIR}/10_correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print('Correlation Matrix:')
print(corr_matrix.round(3))
klar_corr = corr_matrix['KLAR'].drop('KLAR').sort_values(ascending=False)
print(f'\nKLAR highest correlation: {klar_corr.index[0]} ({klar_corr.iloc[0]:.3f})')
print(f'KLAR lowest correlation:  {klar_corr.index[-1]} ({klar_corr.iloc[-1]:.3f})')

## 4.3 — Competitor Scorecard

In [ ]:
# Scores: 1 = poor, 5 = excellent
scorecard = pd.DataFrame({
    'Metric': [
        'Revenue Growth (YoY)',
        'P/S Multiple (lower = cheaper)',
        'Profitability',
        'User Scale',
        'Stock Performance (since Sep 10)',
        'Regulatory Risk',
        'AI / Tech Investment',
        'Credit Quality',
    ],
    'KLAR': [5, 4, 1, 5, 1, 2, 5, 3],
    'AFRM': [5, 3, 3, 3, 4, 3, 3, 4],
    'PYPL': [2, 5, 5, 5, 2, 4, 3, 5],
    'SQ':   [2, 3, 2, 3, 3, 3, 3, 3],
})

fig, ax = plt.subplots(figsize=(14, 7))
x = np.arange(len(scorecard))
w = 0.2

for i, (ticker, color) in enumerate(TICKER_COLORS.items()):
    offset = (i - 1.5) * w
    ax.bar(x + offset, scorecard[ticker], width=w,
           color=color, label=ticker, alpha=0.85, zorder=3)

ax.set_xticks(x)
ax.set_xticklabels(scorecard['Metric'], rotation=25, ha='right', fontsize=10)
ax.set_yticks([1, 2, 3, 4, 5])
ax.set_yticklabels(['Poor', 'Below Avg', 'Average', 'Good', 'Excellent'])
ax.set_ylim(0, 6)
ax.set_title('BNPL Competitor Scorecard (1-5 Rating per Metric)',
             color='white', fontsize=13, loc='left', pad=14)
ax.legend(loc='upper right', framealpha=0.2, fontsize=10)
ax.grid(True, axis='y', alpha=0.3)

totals = {t: scorecard[t].sum() for t in ['KLAR', 'AFRM', 'PYPL', 'SQ']}
plt.figtext(0.01, 0.01,
    'Total Scores:  ' + '   '.join([f'{t}: {v}/40' for t, v in totals.items()]),
    color=COLORS['muted'], fontsize=9)

plt.tight_layout()
plt.savefig(f'{VIS_DIR}/11_competitor_scorecard.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Total Scores: {totals}')

## 4.4 — Written Verdict

In [ ]:
verdict = """
=============================================================================
  BNPL UNDER THE MICROSCOPE — FINAL ANALYTICAL VERDICT
=============================================================================

DATA SUMMARY (as of Feb 12, 2026)
----------------------------------
KLAR stock: $19.75  (-51% from IPO price, -65% from ATH of $57.20)
Analyst consensus: Buy | 14 analysts | Avg target $43.29 (+119% upside)
Q3 2025: Revenue $903M (+26% YoY), GMV $32.7B (+25%), Net Loss -$95M
US GMV growth: +43% YoY
Merchants: 850K (+38% YoY). Active users: 114M
Market cap: $7.82B | P/S ratio: ~2.2x

WHAT THE DATA SHOWS
--------------------
1. KLARNA IS FUNDAMENTALLY STRONG but market-structurally broken.
   Revenue growing 26% YoY at $3.5B run rate is genuinely impressive.
   But it IPO'd into peak AI bubble enthusiasm, and the correction has
   been brutal. The business has not deteriorated as much as the stock.

2. THE PROFITABILITY GAP is the real story.
   Q3 net loss -$95M vs prior year +$12M profit = regression.
   ARPU is declining as merchant count surges. Klarna is adding
   volume but not monetising it efficiently. The debit card and
   banking products must drive ARPU recovery to justify the multiple.

3. CONSUMER RISK IS REAL AND GROWING.
   41% of BNPL users missed payments in 2025 (up from 34% in 2024).
   61% are subprime. 63% stack multiple loans simultaneously.
   The CFPB and NY Fed are building groundwork for mandatory credit
   bureau reporting which would force tighter underwriting.

4. AFFIRM IS OUTPERFORMING KLARNA on the public market.
   AFRM: +12% since KLAR IPO. KLAR: -51%. Same sector, same macro.
   Affirm's focus on fewer larger merchants with interest-bearing
   loans creates cleaner more predictable revenue.

VERDICT
--------
Bull:  $43-55 (12-month), driven by Q4 earnings surprise + ARPU recovery
Base:  $20-35 near-term range-bound pending litigation clarity
Bear:  $10-14 if macro deteriorates and BNPL defaults surface

Not financial advice. All data sourced from public filings and reports.
=============================================================================
"""

print(verdict)

with open(f'{PROC_DIR}/final_verdict.txt', 'w') as f:
    f.write(verdict)
print('Saved: data/processed/final_verdict.txt')

In [ ]:
# Summary of all outputs
print('All Visuals Generated:')
print('=' * 50)
for i, f in enumerate(sorted(os.listdir(VIS_DIR)), 1):
    size_kb = os.path.getsize(f'{VIS_DIR}/{f}') / 1024
    print(f'  {i:02d}. {f:<40} {size_kb:.0f}KB')

print('\nProcessed Data Files:')
print('=' * 50)
for f in sorted(os.listdir(PROC_DIR)):
    print(f'      {f}')

print('\nAll 4 phases complete!')
print('Next steps:')
print('  1. Push to GitHub with README.md')
print('  2. Export key charts as LinkedIn carousel images')
print('  3. Write LinkedIn post: hook -> findings -> CTA')
print('  4. Drop GitHub link in comments, not post body')